In [0]:
#%tensorflow_version 2.x  # Run this if on Colab!!
import tensorflow as tf

In [0]:
tf.__version__  # make sure it's 2.0!

In [0]:
import numpy as np
import os

In [0]:
# If you want to get datasets.py via google drive...

#from google.colab import drive
#drive.mount('/content/drive')

In [0]:
# drive thing part 2
#os.chdir("drive/My Drive/Colab Notebooks")

In [0]:
from datasets import MNISTDataset

In [0]:
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

data = MNISTDataset(train_images.reshape([-1, 784]), train_labels, test_images.reshape([-1, 784]), test_labels, 128)

In [0]:
train_steps = 1000
lr = 0.1

W = tf.Variable(np.zeros([784, 10]).astype(np.float32))
b = tf.Variable(np.zeros(10, dtype=np.float32))

# first change: set up log dir and file writer(s)
import time
logdir = os.path.join("logs", "linear" + str(time.time()))
train_writer = tf.summary.create_file_writer(os.path.join(logdir, "train"))
test_writer = tf.summary.create_file_writer(os.path.join(logdir, "test"))

In [0]:
for step in range(train_steps):
    img_batch, lbl_batch = data.next_batch()
    with tf.GradientTape() as tape:
        logits = tf.matmul(img_batch, W) + b
        xent = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=lbl_batch))
        
    grads = tape.gradient(xent, [W, b])
    W.assign_sub(lr * grads[0])
    b.assign_sub(lr * grads[1])
    
    # change #2: log this stuff every time step (rather wasteful)
    with train_writer.as_default():
        tf.summary.scalar("loss", xent, step=step)
        tf.summary.histogram("logits", logits, step=step)
        tf.summary.histogram("weights", W, step=step)
    
    if not step % 100:
        preds = tf.argmax(logits, axis=1, output_type=tf.int32)
        acc = tf.reduce_mean(tf.cast(tf.equal(preds, lbl_batch), tf.float32))
        
        # change #3: log this only once every 100 steps
        with train_writer.as_default():
            tf.summary.scalar("accuracy", acc, step=step)
            tf.summary.image("input", tf.reshape(img_batch, [-1, 28, 28, 1]), step=step)
            
        test_preds = tf.argmax(tf.matmul(data.test_data, W) + b, axis=1, output_type=tf.int32)
        test_acc = tf.reduce_mean(tf.cast(tf.equal(test_preds, data.test_labels), tf.float32))
        with test_writer.as_default():
            tf.summary.scalar("accuracy", test_acc, step=step)
        
        print("Training Loss: {} Accuracy: {}".format(xent, acc))
        print("Test accuracy: {}\n".format(test_acc))

In [0]:
# then load/run tensorboard

%load_ext tensorboard

In [0]:
%tensorboard --logdir logs